In [1]:
_ = '''
transaction = contract.newEvent({
    'currencyPair': 'XTZ-USD',
    'targetDynamics': 1000000,
    'betsCloseTime': int(time.time()) + 3600,
    'measurePeriod': 3600,
    'liquidityPercent': 10000,
}).as_transaction()

transaction.json_payload()#.hash()
'''

### Juster-Maker:
Tool allowing to run events in cycle

In [2]:
from pytezos import pytezos
import time
from pprint import pprint
from utility import repeat_until_succeed
from bulk_sender import BulkSender
from event_emitter import EventCreationEmitter


# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [8]:
_ = '''
CONFIG = {
    'events': [],
    'juster_address': 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
}
'''

In [3]:
address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
client = pytezos.using(
    key='../2021-05-27-tests-modelling/test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json'
)
contract = client.contract(address)

In [12]:
import time
time.time()

1626711356.0427566

In [20]:
from datetime import datetime

def date_to_timestamp(date, time_format='%Y-%m-%d %H:%M:%S'):
    """ Converts date to timestamp using by default
        '%Y-%m-%d %H:%M:%S' format """

    return datetime.strptime(date, time_format).timestamp()

date_to_timestamp('2021-07-19 20:00:00')

1626714000.0

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626712726, 'measurePeriod': 900, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626712726, 'measurePeriod': 900, 'liquidityPercent': 10000}


### TODO:
- starts_from parameter for events creators so it would be possible to bind events to the time slots

In [9]:
from asyncio import Queue

# TODO: what would happen if queue overflow?
MAX_QUEUE_SIZE = 50

operations_queue = Queue(MAX_QUEUE_SIZE)

events_params = [
    {
        'currency_pair': currency_pair,
        # 'first_at': 
        'target_dynamics': 1_000_000,
        'bets_period': 900,
        'measure_period': 900,
        'liquidity_percent': 10_000,
        'expiration_fee': 100_000,
        'measure_start_fee': 100_000
    } for currency_pair in ['XTZ-USD', 'BTC-USD']
]

In [10]:
# for each event_params EventCreationEmitter is created:
emitters = [
    EventCreationEmitter(
        period=params['bets_period'],
        contract=contract,
        operations_queue=operations_queue,
        event_params=params)
    for params in events_params
]

emitters.append(
    BulkSender(period=60, client=client, operations_queue=operations_queue)
)

In [11]:
for emitter in emitters:
    emitter.run()

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626710926, 'measurePeriod': 900, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626710926, 'measurePeriod': 900, 'liquidityPercent': 10000}
making bulk of 2 operations
signed, result hash: oo1f5c1TAD8UFNeFpRLJYyBxMRcuS21t8hdWNBNS4MCn9XqJeYE
making bulk of 0 operations
catched <class 'pytezos.rpc.node.RpcError'> in sign: ('"Assert_failure '
 'src/proto_009_PsFLoren/lib_protocol/operation_repr.ml:203:6"',)
> /home/fortune/Documents/DeFiProjects/Juster/juster-core/projects/2021-07-19-juster-maker-proto/bulk_sender.py(39)sign()
     37             # operations back to the queue, if not: raise this e
     38             import pdb; pdb.set_trace()
---> 39             raise e
     40 
     41 

ipdb> 
created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'ta

In [26]:
pytezos


Properties
.key		tz1grSQDByRpnVs7sPtaprNZRp531ZKz6Jmm
.shell		['https://api.tez.ie/rpc/florencenet']
.block_id	head

Helpers
.account()
.activate_account()
.activate_protocol()
.bake_block()
.balance()
.ballot()
.bulk()
.check_message()
.contract()
.delegation()
.double_baking_evidence()
.double_endorsement_evidence()
.endorsement()
.endorsement_with_slot()
.failing_noop()
.now()
.operation()
.operation_group()
.origination()
.proposals()
.reveal()
.seed_nonce_revelation()
.sign_message()
.transaction()
.using()
.wait()

created newEvent transaction with parameters: {'currencyPair': 'XTZ-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626715427, 'measurePeriod': 900, 'liquidityPercent': 10000}
created newEvent transaction with parameters: {'currencyPair': 'BTC-USD', 'targetDynamics': 1000000, 'betsCloseTime': 1626715427, 'measurePeriod': 900, 'liquidityPercent': 10000}


### TODO: need to test all this code before running in florencenet

In [7]:
for emitter in emitters:
    emitter.stop()